In [ ]:
%pip install requests
%pip install beautifulsoup4
%pip install brotli     #Decompression

In [ ]:
%pip install --upgrade pip


In [ ]:
%pip install brotli

In [ ]:
%pip install selenium

In [ ]:
from bs4 import BeautifulSoup
import requests 
import urllib, urllib.request
import io
import base64
import re
import brotli
import gzip

In [ ]:
#Selenium setup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import SessionNotCreatedException
import traceback

In [ ]:
def getHTML2(url):
    user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.102 Safari/537.36"
    #result = urllib.request.urlopen(url)
    result = requests.get(url, timeout=20, headers={ "User-agent" : user_agent, 
                                                 "Accept-Language": "en-US,en;q=0.5",
                                                 "Accept-Encoding": "gzip, deflate, br",
                                                 "Referer": "https://www.google.com/"})
    print(result)
    #print(result)
    #
    #htmlResult = result.text
    #print(htmlResult)
    #
    ##BSoup engine
    #soup = BeautifulSoup(htmlResult, "html.parser") #PARALLELISM POSSIBLE
    if result.status_code == 200:
        # Comprobar si la respuesta está comprimida
        #print("Content-Encoding header:", result.headers.get('Content-Encoding'))
        if result.headers.get('Content-Encoding') == 'bra':  #br si está comprimido
            # Descomprimir el contenido
            compressed_data = result.content
            decompressed_data = brotli.decompress(compressed_data)
            # Parsear el HTML utilizando BeautifulSoup
            htmlResult = decompressed_data.decode('utf-8')
        else:
            htmlResult = result.text

        # Imprimir el HTML
        #print(htmlResult)

        #BSoup engine
        soup = BeautifulSoup(htmlResult, "html.parser")
        return soup
    else:
        print("Error al obtener la página:", result.status_code)

In [28]:
base_url = "https://www.alpacainfo.com/search/alpacas"   #Base url


In [94]:
def getHTML(url, cur_param=None):
    ##Make that you are a typical user
    try:
        user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.102 Safari/537.36"
        result = requests.get(url, params=cur_param, timeout=20, headers={ "User-agent" : user_agent, 
                                                 "Accept-Language": "en-US,en;q=0.5",
                                                 "Accept-Encoding": "gzip, deflate, br",
                                                 "Referer": "https://www.google.com/"})
    except Exception as e:
        return None
    if result.status_code == 200:
        if result.headers.get('Content-Encoding') == 'bra':  #br in case it is compressed
            # Descomprimir el contenido
            compressed_data = result.content
            decompressed_data = brotli.decompress(compressed_data)
            # Parsear el HTML utilizando BeautifulSoup
            htmlResult = decompressed_data.decode('utf-8')
        elif result.headers.get('Content-Encoding') == 'gzipa':  #br in case it is compressed
            # Descomprimir el contenido
            compressed_data = result.content
            decompressed_data = gzip.decompress(compressed_data)
            # Parsear el HTML utilizando BeautifulSoup
            htmlResult = decompressed_data.decode('utf-8')
        else:
            htmlResult = result.text

        soup = BeautifulSoup(htmlResult, "html.parser")
        return soup
    else:
        print("Error al obtener la página:", result.status_code, cur_param)
        return None

In [55]:
##FUNCIONAAAAAAA
# Configuración de Selenium
service = webdriver.chrome.service.Service("../WebScrapperAlpacaInfo/chromedriver-win64/chromedriver.exe")
driver = webdriver.Chrome(service=service)

# Número de páginas a consultar
num_pages = 40

# Lista para almacenar los primeros nombres de cada página
first_name_links = []

for page_num in range(1, num_pages + 1):
    # Construir la URL de la página actual
    url = f"{base_url}#?page={page_num}"

    # Abrir la página actual en el navegador
    driver.get(url)

    # Esperar a que el botón Next esté presente en el DOM
    if page_num<40: #LastPage
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "//a[@title='Next']")))

    # Obtener el contenido HTML de la página actual después de esperar
    html_content = driver.page_source

    # Utilizar BeautifulSoup para analizar el HTML
    soup = BeautifulSoup(html_content, 'html.parser')

    # Encontrar los nombres y los links
    name_elements = soup.find_all('td', class_='first-column')
    for name_element in name_elements:
        link = name_element.find('a')
        first_name_links.append({'name': link.text.strip(), 'link': link['href']})


    # Hacer clic en el botón Next después de que esté presente y esperar a que la página se cargue completamente
    if page_num<40: #LastPage
        next_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//a[@title='Next']")))
        next_button.click()
        WebDriverWait(driver, 10).until(EC.staleness_of(next_button))
 
# Imprimir los primeros nombres de cada página
# Imprimir los nombres y enlaces de cada página
for index, shop in enumerate(first_name_links, start=1):
    print(f"Página {index}: Name: {shop['name']:<40} | URL: {shop['link']}")

# Cerrar el navegador
driver.quit()

##FUNCIONAAAAA

Página 1: Name: PRTI PRESIDIO'S MUSCAT                   | URL: /search/alpaca/32776638-1;jsessionid=CBD59A6B23D297AEE9EB5B2F52B616AE.1b
Página 2: Name: BOGART'S BIANCA                          | URL: /search/alpaca/31151993-1;jsessionid=CBD59A6B23D297AEE9EB5B2F52B616AE.1b
Página 3: Name: PURPLE POST'S JERSEY DEVIL               | URL: /search/alpaca/32420791-1;jsessionid=CBD59A6B23D297AEE9EB5B2F52B616AE.1b
Página 4: Name: WINDY'S ISREAL                           | URL: /search/alpaca/31967662-1;jsessionid=CBD59A6B23D297AEE9EB5B2F52B616AE.1b
Página 5: Name: ROSEHAVEN'S PERUVIAN SILVER CROWN        | URL: /search/alpaca/30470309-1;jsessionid=CBD59A6B23D297AEE9EB5B2F52B616AE.1b
Página 6: Name: WINDY'S WAYNE                            | URL: /search/alpaca/31967648-1;jsessionid=CBD59A6B23D297AEE9EB5B2F52B616AE.1b
Página 7: Name: WINDY'S OSCAR                            | URL: /search/alpaca/35061922-1;jsessionid=CBD59A6B23D297AEE9EB5B2F52B616AE.1b
Página 8: Name: BIG TIMBER KAHUNA'S JOSEF

In [57]:
info_name_address_phone_website=[]           ##Get all the first info appeared in alpaca info page

for link in first_name_links:
    ##In two lines we cut the url and get the number for appending to the url
    pageInfoURL = base_url.replace("/search/alpacas", "") + link["link"]
    print(pageInfoURL)
    soup = getHTML(pageInfoURL)
    if(soup==None):         ##In case we cannot acces that number page of the table
        continue
    #print(soup)
    info = soup.find("div", class_="contact-info")
    if(info==None):
        continue
    listInfo=[]
    listInfo.append(link['name'])
    listInfo.append(info.find("div", class_="address").text)
    listInfo.append(info.find("div", class_="phone").text)
    website=info.find("div", class_="website")
    listInfo.append(website.find("a")["href"])
    #print(listInfo)
    info_name_address_phone_website.append(listInfo)

print(info_name_address_phone_website)

[["PRTI PRESIDIO'S MUSCAT", '1154 County Road 4360DecaturTX76234United States', '940-393-1483', 'http://www.alpacasamerica.com'], ["BOGART'S BIANCA", '4360 Indian Creek RdLincolnCA95648United States', '530-218-6255', 'http://www.colusariversidealpacas.com'], ["PURPLE POST'S JERSEY DEVIL", '61 White Pine RdChesterfieldNJ08515United States', '609-947-8269', 'http://www.windyfarmalpacas.com'], ["WINDY'S ISREAL", '61 White Pine RdChesterfieldNJ08515United States', '609-947-8269', 'http://www.windyfarmalpacas.com'], ["ROSEHAVEN'S PERUVIAN SILVER CROWN", '61 White Pine RdChesterfieldNJ08515United States', '609-947-8269', 'http://www.windyfarmalpacas.com'], ["WINDY'S WAYNE", '61 White Pine RdChesterfieldNJ08515United States', '609-947-8269', 'http://www.windyfarmalpacas.com'], ["WINDY'S OSCAR", '61 White Pine RdChesterfieldNJ08515United States', '609-947-8269', 'http://www.windyfarmalpacas.com'], ["BIG TIMBER KAHUNA'S JOSEF", '29400 SW Heater RdSherwoodOR97140United States', '', 'http://www.b

In [39]:
path="output.txt"

In [43]:
def find_email_raw_text(htmlText):
    with open(path, 'wb') as file:  # Abre el archivo en modo binario
        file.write(htmlText.encode('utf-8'))  # Codifica el texto como UTF-8 antes de escribirlo
    emails = re.findall(r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b', htmlText)
    if not emails:
        return None
    return emails

In [41]:
##Decode the email and get by emailto
def decode_email(encoded_email):
    emailriddlerarray = re.findall(r'\d+', encoded_email)  # Extraer los números del script
    decoded_email = ''.join(chr(int(code)) for code in emailriddlerarray)  # Decodificar los números
    return decoded_email


def find_by_emailto(soupHTML):
    anchor_tags = soupHTML.find_all('a')

    # Extract email addresses from href attributes
    emails = []
    for tag in anchor_tags:
        href = tag.get('href')
        if href:
            if href.startswith('mailto:'):
                email = href[len('mailto:'):]
                emails.append(email)
            elif re.search(r'encrypted_email', href):
                encoded_email = re.search(r'encrypted_email=(.*)', href)
                if encoded_email:
                    decoded_email = decode_email(encoded_email.group(1))
                    emails.append(decoded_email)
            elif re.search(r'\/\*(.*?)\*\/', href):  # Expresión regular que coincide con el patrón específico del script cifrado
                encoded_email = re.search(r'\/\*(.*?)\*\/', href)
                if encoded_email:
                    decoded_email = decode_email(encoded_email.group(1))
                    emails.append(decoded_email)
            else:
                matches = re.findall(r'[\w\.-]+@[\w\.-]+', href)  # Buscar direcciones de correo electrónico
                emails.extend(matches)
    if not emails:
        return None
    
    return emails


In [95]:
##Get all the contact links 

contactLinks=[]    
def find_contact_page():

    for website in info_name_address_phone_website:
        # Encontrar todas las etiquetas de ancla
        if website[3]=="http://http:www.aaalpacafarmandstore":
            website[3]="https://www.aaalpacafarmandstore.com"
        if website[3]!='':
            try:
                print(website[3])
                soup = getHTML(website[3])
                anchor_tags = soup.find_all('a')

                # Definir patrones para identificar enlaces de página de contacto
                contact_page_patterns = [
                    re.compile(r'contact', re.IGNORECASE),
                    re.compile(r'contact\s*us', re.IGNORECASE),  # Puede tener espacios entre "contact" y "us"
                    re.compile(r'contactus', re.IGNORECASE),     # Puede estar escrito como una palabra
                    re.compile(r'get\s*in\s*touch', re.IGNORECASE),  # Puede tener espacios entre "get", "in" y "touch"
                    re.compile(r'getintouch', re.IGNORECASE),   # Puede estar escrito como una palabra
                    # Puedes agregar más patrones según sea necesario
                ]

                # Buscar enlaces que contengan los patrones de la página de contacto
                contact_page_links = []
                for tag in anchor_tags:
                    href = tag.get('href')
                    if href:
                        for pattern in contact_page_patterns:
                            if pattern.search(href) and href not in contact_page_links:
                                contact_page_links.append(href)
                                break

                # Imprimir los enlaces de la página de contacto encontrados
                for idx, link in enumerate(contact_page_links):
                    if link[0]!='/' and not("http" in link):
                        contact_page_links[idx] ='/'+link
                print(contact_page_links)
                contactLinks.append(contact_page_links)
            except Exception as e:
                contactLinks.append([])
        else:
            contactLinks.append([])
    print(contactLinks)
find_contact_page()

http://www.alpacasamerica.com
['/contact']
http://www.colusariversidealpacas.com
['/contact-us.php']
http://www.windyfarmalpacas.com
['/contact']
http://www.windyfarmalpacas.com
['/contact']
http://www.windyfarmalpacas.com
['/contact']
http://www.windyfarmalpacas.com
['/contact']
http://www.windyfarmalpacas.com
['/contact']
http://www.bigtimberalpacas.com
['/contact']
http://www.alpacas.wordpress.com
[]
http://www.alpacausa.com
['/contact']
http://www.skyfiberranch.com
[]
http://www.islandalpaca.com
['http://islandalpaca.com/contact_island_alpaca.php']
http://www.islandalpaca.com
['http://islandalpaca.com/contact_island_alpaca.php']
http://www.islandalpaca.com
['http://islandalpaca.com/contact_island_alpaca.php']
http://www.islandalpaca.com
['http://islandalpaca.com/contact_island_alpaca.php']
http://www.islandalpaca.com
['http://islandalpaca.com/contact_island_alpaca.php']
http://www.islandalpaca.com
['http://islandalpaca.com/contact_island_alpaca.php']
http://www.islandalpaca.com
['h

In [96]:
print(len(first_name_links),len(info_name_address_phone_website),len(contactLinks))

399 399 399


In [ ]:
gmails = []
farm=info_name_address_phone_website
for i in range(len(info_name_address_phone_website)):
    inserted=False
    for link in contactLinks[i]:
        if (farm[i][3]==''):
            continue
        elif link=='':
            contactURL=farm[i][3]
        elif not ("http" in link):
            contactURL=farm[i][3]+link
        elif "http" in link:
            contactURL=link
        print(contactURL)
        soup=getHTML(contactURL)
        if(soup==None):
            continue

        gmail=soup.find("input",{"id":"Email"})         ##En caso sea /contact que es la mayoria
        if(gmail==None):
            gmail=find_email_raw_text(soup.getText())
            if(gmail==None):
                print("Gmail not found")
                continue
            gmails.append(gmail)
            inserted=True
            break
        gmail_value = gmail.get("value")    
        gmails.append(gmail_value)
        inserted=True
        break
    
    if inserted==False:
        ##Search in the main page
        print("-->", farm[i][3])
        if (farm[i][3]==''):  #No page
            soup=None
        else:
            soup=getHTML(farm[i][3])
        if(soup==None):
            gmails.append(None)
            print("Not found page")
        else:
            gmail=find_by_emailto(soup)
            gmails.append(gmail)

print(gmails)

In [101]:
import pandas as pd

headers= ["Nombre","Dirección", "Número", "Página web", "Gmail"]

df_name_href = pd.DataFrame()

data_names_href = []
for link in first_name_links:
    #data_names_href.append({"Nombres": link.span.text, "URL": link["link"]})
    data_names_href.append({"URL": link["link"]})
df_nombres_href =pd.DataFrame(data_names_href)

df_firstInfo = pd.DataFrame(info_name_address_phone_website, columns=["Name","Dirección", "Celular", "Sitio web"])

df_email = pd.DataFrame(gmails, columns=["Gmail"])


df_final= pd.concat([df_nombres_href, df_firstInfo,df_email], axis=1)
df_final.index += 1
df_final


excel_file = "alpacaInfoScrappingCompletedjustScrap.xlsx"
df_final.to_excel(excel_file, index=False)

In [102]:
df_final[]

,URL,Name,Dirección,Celular,Sitio web,Gmail
1,/search/alpaca/32776638-1;jsessionid=CBD59A6B2...,PRTI PRESIDIO'S MUSCAT,1154 County Road 4360DecaturTX76234United States,940-393-1483,http://www.alpacasamerica.com,nancypulse@gmail.com
2,/search/alpaca/31151993-1;jsessionid=CBD59A6B2...,BOGART'S BIANCA,4360 Indian Creek RdLincolnCA95648United States,530-218-6255,http://www.colusariversidealpacas.com,None
3,/search/alpaca/32420791-1;jsessionid=CBD59A6B2...,PURPLE POST'S JERSEY DEVIL,61 White Pine RdChesterfieldNJ08515United States,609-947-8269,http://www.windyfarmalpacas.com,jkarmiger@gmail.com
4,/search/alpaca/31967662-1;jsessionid=CBD59A6B2...,WINDY'S ISREAL,61 White Pine RdChesterfieldNJ08515United States,609-947-8269,http://www.windyfarmalpacas.com,jkarmiger@gmail.com
5,/search/alpaca/30470309-1;jsessionid=CBD59A6B2...,ROSEHAVEN'S PERUVIAN SILVER CROWN,61 White Pine RdChesterfieldNJ08515United States,609-947-8269,http://www.windyfarmalpacas.com,jkarmiger@gmail.com
...,...,...,...,...,...,...
395,/search/alpaca/35599449-1,Ten Seven Acres' Peruvian Finnegan,107 Galchutt AveGalchuttND58075United States,701-425-1822,http://www.tensevenacres.com,None
396,/search/alpaca/36039883-1,XANADU Enchanter,11785 SW River RdHillsboroOR97123United States,503-799-7789,http://www.pacaparadise.com,steve.alpacas@gmail.com
397,/search/alpaca/32819373-1,BAV'S DELORIAN,256 Coulter DrNew MarketAL35761United States,256-975-1224,,None
398,/search/alpaca/32794236-1,STEEPLE VIEW DORIAN'S ECKO,256 Coulter DrNew MarketAL35761United States,256-975-1224,,None


In [ ]:
print(gmails)

In [ ]:
print(firstLinks)